In [3]:
#FINAL VERSION#

import csv
import serial
from NeuroPy import NeuroPy
import time
import cv2
import threading
import os

class Gather(threading.Thread):
    def __init__(self,dataset,x,totalTime,samplingRate,delayTime):
        threading.Thread.__init__(self)
        self._stop_event=threading.Event()
        self.dataset=dataset
        self.x=x
        self.totalTime=totalTime
        self.samplingRate=samplingRate
        self.delayTime=delayTime
        self.stopGather=False
        
    def run(self):
        gatherData(self.dataset,self.x,self.totalTime,self.samplingRate,self.delayTime,self)
    
    def pause(self):
        self._stop_event.wait()
    
    def resume(self):
        self._stop_event.clear()
        
    def is_paused(self):
        return self._stop_event.is_set()
    
    def stop(self,timeout=None):
        self._stop_event.set()
        self.stopGather=True
        threading.Thread.join(self,timeout)
        
    def is_stopped(self):
        return self._stop_event.is_set() and self.stopGather

def do_every(function,period,number,gather,*args):
    def g_tick():
        t=time.time()
        count = 0
        while True:
            count += 1
            yield max(t + count*period - time.time(),0)
    g=g_tick()
    for i in range(number):
        function(*args)
        time.sleep(next(g))
        if gather.is_stopped():
                break

def data(datapoint):
    datapoint.append(x.rawValue)

def gatherData(dataset,x,totalTime,samplingRate,delayTime,gather):
    counter=1
    #start=time.time()
    #time.sleep(delayTime)
    print("Test Start!")
    for c in range (0,stimulusNum):
        #print(time.time()-start)
        datapoint=[]
        #start=time.time()
        do_every(data,1./samplingRate,(int)(totalTime/(1./samplingRate)),gather,datapoint)
        if gather.is_stopped():
            print("Data Gathering Ended Early")
            break
        dataset.append(datapoint)
        #print("Gather Time:",time.time()-start)
        if c%2==0:
            print ("Positive Face", counter)
        else:
            print ("Negative Face",counter)
            counter=counter+1

def tick(period):
    t=time.time()
    count = 0
    while True:
        count += 1
        yield max(t + count*period - time.time(),0)
#def showFrame(cv2,frame):

ModuleNotFoundError: No module named 'NeuroPy'

In [ ]:
filename='Ray Xinrui Wang_4.csv'
windowName="Test"
comNum="COM7"
dirPath="C:\\Users\\panda\\Cogs189 Pictures\\Final Test\\Test Part 4"
fileName="\\Slide"
extension=".png"
stimulusTime=1
transition=1
countdown=1
totalCountdown=5
stimStart=6
totalTime=stimulusTime+transition
samplingRate=250.0
stimulusNum=50
delayTime=10
ms=1000
videoDelay=0.002
dataset=[]


x=NeuroPy(comNum,9600)
t=Gather(dataset,x,totalTime,samplingRate,delayTime+totalCountdown)
list=os.listdir(dirPath)
threadCount=0

try:
    x.start()
except serial.SerialException:
    serial.Serial().close()
    x.start()

print "Connect Successful"



print "Video Start!"

cv2.namedWindow(windowName, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

for i in range(0,len(list)):
    start=time.time()
    frame = cv2.imread(dirPath+fileName+str(i+1)+extension)
    if frame is not None and frame.size != 0:
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        frame= cv2.resize(frame,(1440, 900),fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
        cv2.imshow(windowName, frame)
        if(i==0):
            waitTime=delayTime
        elif(i>=1 and i<=5):
            waitTime=countdown
        elif(i==6):
            waitTime=(transition/2.0)
        elif(i%2==0):
            waitTime=stimulusTime
        else:
            waitTime=transition
    else:
        break
    waitTime=waitTime-(time.time()-start)-videoDelay
    if(i!=0):
        wait=tick(waitTime)
        time.sleep(next(wait))
    else:
        cv2.waitKey((int)(waitTime*ms))
    #if cv2.waitKey((int)(next(tick)*ms)) & 0xFF == ord('q'):
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print "Stopping"
        t.stop()
        break
        
    if(threading.activeCount()!=threadCount and i>=6):
        if(threadCount==0):
            #Start the thread
            t.start() 
            threadCount=threading.activeCount()
        else:
            break
    #print(time.time()-start)
    #print("Video Sleep:",time.time()-start)

    
cv2.destroyAllWindows() 
while(True):
    if(threading.activeCount()==threadCount):
        time.sleep(1)
    else:
        with open(filename,'wb') as outfile:
            outfileWrite = csv.writer(outfile)
            outfileWrite.writerows(dataset)
        x.stop()
        break

print "Data Gathering Finished"


In [ ]:
## filename='testdata.csv'
frequency=["raw","delta","theta","lowAlpha","highAlpha","lowBeta","highBeta","lowGamma","midGamma"]
peakFreq=[]
comNum="COM6"
dirPath="C:\\Users\\panda\\Cogs189 Pictures\\Test 3.0"
fileName="\\Slide"
extension=".png"
stimulusTime=1
transition=1
introTime=2
countdownTime=1
totalTime=stimulusTime+transition
samplingRate=250.0
stimulusNum=200
stimFrames=3
ms=1000
gatherDelay=-0.01
firstStim=2
dataset=[]
gather_T=[]
frame_T=[]

x=NeuroPy(comNum,9600)
list=os.listdir(dirPath)
fileNum=2
threadIndex=0
counter=1

try:
    x.start()
except serial.SerialException:
    serial.Serial().close()
    x.start()

print "Connect Successful"

print "Video Start!"
#Output the intro slide
introSlides=fileNameIter(dirPath,fileName,extension,1,firstStim)
intro(cv2,introSlides,firstStim,introTime,countdownTime)
#Start the thread
while(True):
    frames=fileNameIter(dirPath,fileName,extension,fileNum,stimFrames)
    #gather_T.append(Gather(dataset,x,totalTime,samplingRate))
    frame_T.append(Frame(cv2,frames,stimulusTime,transition))
    #gather_T[threadIndex].start()
    frame_T[threadIndex].start()
    threadCount=threading.activeCount()

    
    if(threading.activeCount()!=threadCount):
        break
        
    #gather_T[threadIndex].join()
    frame_T[threadIndex].join()
    
    if(fileNum>=stimulusNum):
        break

    if threadIndex%2==0:
        print "Positive Face",counter
    else:
        print "Negative Face",counter
        counter=counter+1
        
    threadIndex=threadIndex+1
    fileNum=fileNum+2
    
    
    #print(time.time()-start)
    
cv2.destroyAllWindows() 

with open(filename,'wb') as outfile:
    outfileWrite = csv.writer(outfile)
    outfileWrite.writerows(dataset)

x.stop()


In [ ]:
import csv
import serial
from NeuroPy import NeuroPy
import time
import cv2
import threading
import os

class Gather(threading.Thread):
    def __init__(self,dataset,x,totalTime,samplingRate):
        threading.Thread.__init__(self)
        self._stop_event=threading.Event()
        self.dataset=dataset
        self.x=x
        self.totalTime=totalTime
        self.samplingRate=samplingRate
        self.stopGather=False
        
    def run(self):
        gatherData(self.dataset,self.x,self.totalTime,self.samplingRate,self)
    
    def pause(self):
        self._stop_event.wait()
    
    def resume(self):
        self._stop_event.clear()
        
    def is_paused(self):
        return self._stop_event.is_set()
    
    def stop(self,timeout=None):
        self._stop_event.set()
        self.stopGather=True
        threading.Thread.join(self,timeout)
        
    def is_stopped(self):
        return self._stop_event.is_set() and self.stopGather

def do_every(function,period,number,gather,*args):
    def g_tick():
        t=time.time()
        count = 0
        while True:
            count += 1
            yield max(t + count*period - time.time(),0)
    g=g_tick()
    for i in range(number):
        function(*args)
        time.sleep(next(g))
        if gather.is_stopped():
            break

def data(datapoint):
    datapoint.append(x.rawValue)

def gatherData(dataset,x,totalTime,samplingRate,gather):
    counter=1
    datapoint=[]
    do_every(data,1./samplingRate,(int)(totalTime/(1./samplingRate)),gather,datapoint)
    if gather.is_stopped():
        print("Data Gathering Ended Early")
        gather.stop()
    dataset.append(datapoint)
    


    




In [ ]:
class Frame(threading.Thread):
    def __init__(self,cv2,frames,stimulusTime,transition):
        threading.Thread.__init__(self)
        self._stop_event=threading.Event()
        self.cv2=cv2
        self.frames=frames
        self.stimulusTime=stimulusTime
        self.transition=transition
        
    def run(self):
        showStimulus(self.cv2,self.frames,self.stimulusTime,self.transition,self)
    
    def stop(self,timeout=None):
        self._stop_event.set()
        threading.Thread.join(self,timeout)
        
    def is_stopped(self):
        return self._stop_event.is_set()

def showFrame(cv2,frameName,waitTime):
    frame=cv2.imread(frameName)
    if frame is not None and frame.size != 0:
        cv2.imshow('Test', frame)
    else:
        return False
    if cv2.waitKey((int)(waitTime*ms)) & 0xFF == ord('q'):
        print "Stopping"
        return False
    return True

def intro(cv2,frames,firstStim,introTime,countdownTime):
    for i in range(firstStim):
        if(i==0):
            waitTime=introTime
        else:
            waitTime=countdownTime
        if(showFrame(cv2,frames[i],waitTime)==False):
            break

def fileNameIter(dirPath,fileName,extension,fileNum,numIter):
    frames=[]
    for i in range(numIter):
        frames.append(dirPath+fileName+str(fileNum+i)+extension)
    return frames
    
def showStimulus(cv2,frames,stimulusTime,transition,thread):
    for i in range(3):
        if(i%2==0):
            waitTime=(int)(transition/2.0)
        else:
            waitTime=stimulusTime
        frames=cv2.imread(frames[0])
        cv2.imshow('Test', frames[0])
        if cv2.waitKey(0) & 0xFF == ord('q'): 
            thread.stop()
            break
        #if(showFrame(cv2,frames[i],waitTime)==False):
            #thread.stop()
            #break
            
            
        

In [ ]:
showStimulus(cv2,frames)

In [ ]:
from threading import Event, Thread
import time

event = Event()

def gatherData():
    for i in range(20):
        print("Gathering Data")
        event.wait()
        print("Finished")
        event.clear()
    
def showVideo():
    for i in range(15):
        print("Showing Video")
        time.sleep(1)
        event.set()

threadsD=[]
threadsV=[]

threadsD.append(Thread(target=gatherData))
threadsV.append(Thread(target=showVideo))
threadsD[0].start()
threadsV[0].start()
threadsD[0].join()
threadsV[0].join()

In [ ]:
from threading import Event, Thread
import time

event = Event()

def gatherData():
    print("Gathering Data")
    event.wait()
    print("Finished")
    event.clear()
    
def showVideo():
    print("Showing Video")
    time.sleep(1)
    event.set()

threadsD=[]
threadsV=[]

for i in range(20):
    threadsD.append(Thread(target=gatherData))
    threadsV.append(Thread(target=showVideo))
    threadsD[i].start()
    threadsV[i].start()
    threadsD[i].join()
    threadsV[i].join()

    
    

In [ ]:
def do_every(period,function,number,*args):
    def g_tick():
        t=time.time()
        count = 0
        while True:
            count += 1
            yield max(t + count*period - time.time(),0)
    g=g_tick()
    for i in range(number):
        time.sleep(next(g))
        function(*args)
    
def gatherData():
    print("Datapoint",time.time())
    #time.sleep(0.006)

start=time.time()
do_every(1./samplingRate,gatherData,(int)(totalTime/(1./samplingRate)))
print(time.time()-start)

In [ ]:
from tkinter import Tk
from timeit import default_timer

def timer():
    return int(default_timer()*1000+.5)
def tick(interval, function, *args):
    root.after(interval-timer() % interval, tick, interval, function, *args)
    function(*args)

def gatherData():
    print "Data Gathered"

interval=4
root=Tk()
root.withdraw()
root.after(interval - timer() % interval, tick, interval, gatherData)
root.mainloop()

In [ ]:
import cv2
import os
import time

list = os.listdir("C:\\Users\\panda\\Cogs189 Pictures\\Test 1")
print(len(list))
#path, dirs, files = next(os.walk("/Cogs189 Pictures/Test 1"))
print "C:/Users/panda/Cogs189 Pictures/Test 1/Slide"+(str)(1)+".png"

for i in range(1,10):
    img = cv2.imread("C:\\Users\\panda\\Cogs189 Pictures\\Test 1\\Slide"+(str)(i)+".png")
    if img is not None and img.size!=0:
        cv2.imshow('Test',img)
        if cv2.waitKey(1000) & 0xFF == ord('q'):
            break
    else:
        print("Load Error")
        break
    
    

cv2.destroyAllWindows()           
        

In [ ]:
#Video v1
import cv2
cap=cv2.VideoCapture('Test 4.0.mp4')

if (cap.isOpened()==False):
    print("Error opening video")

while(cap.isOpened()): 
    # Capture frame-by-frame 
    ret, frame = cap.read() 
    if ret == True: 
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Display the resulting frame 
        cv2.imshow('Frame', frame) 
        cv2.waitKey(0)
    else:
        break
    # Press Q on keyboard to  exit 
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break
cap.release() 
cv2.destroyAllWindows() 

In [ ]:
import threading
import time


def gatherData():
    print("Test Start!")
    #print("Datapoint Gathered")
    for i in range(1):
        for j in range(3):
            print("Datapoint Gathered")
            time.sleep(4)
        print("Next Row")
    

def delayTime():
    time.sleep(5)
    gatherData()
    

t=threading.Thread(target=delayTime,name="Thread1",args=())
t.start()
threadCount=threading.activeCount()
for i in range(500):
    if(threading.activeCount()==threadCount):
        print("Showing Video")
        time.sleep(1)
    else:
        break
    
    
        

In [ ]:
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print "Starting " + self.name
        print_time(self.name, 5, self.counter)
        print "Exiting " + self.name

def print_time(threadName, counter, delay):
    while counter:
        if exitFlag:
            threadName.exit()
        time.sleep(delay)
        print "%s: %s" % (threadName, time.ctime(time.time()))
        counter -= 1

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print "Exiting Main Thread"


In [ ]:
#Old Video Processing Code
video=cv2.VideoCapture('Test 4.0.mp4')


counter=0
while(True):
    ret,frame=video.read()
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if ret==True:
        cv2.imshow('Test', frame)
        movie.append(frame)
    if ret==False:
        break
        
    counter=counter+1
    
video.release() 
cv2.destroyAllWindows() 
print "Video Processed"